In [9]:
from pyalgotrade import strategy
from datetime import datetime
from oanda import fx_lib
import math
import json

class NU3B(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, order_books):
        super(NU3B, self).__init__(feed)
        self.__instrument = instrument
        self.__prices = feed[instrument].getPriceDataSeries()
        self.__order_books = order_books
        self.__sliding_minutes = 1
        self.setUseEventDateTimeInLogs(False)
        self.setDebugMode(False)

    def onBars(self, bars):
        if self.is_lambda_invoke_time() and self.is_restraint_time_zone():
            self.cancel_all_exist_orders()
            self.close_all_open_position()
        if self.is_lambda_invoke_time() and not self.is_restraint_time_zone():
            self.cancel_all_exist_orders()
            ob = order_book[self.getCurrentDateTime().timestamp()]
            buckets = fx_lib.divide_buckets_up_and_down(ob)
            


            # self.info(self.getCurrentDateTime())
            # order_book を確認して注文する

    def is_lambda_invoke_time(self):
        dt = math.floor(self.getCurrentDateTime().timestamp()) - (self.__sliding_minutes * 60)
        return fx_lib.is_order_book_update_time(datetime.fromtimestamp(dt))

    def is_restraint_time_zone(self):
        return self.getCurrentDateTime().hour == 21 or self.getCurrentDateTime().hour == 20

    def cancel_all_exist_orders(self):
        for p in self.getActivePositions():
            if p.entryActive():
                p.cancelExit()
                p.cancelEntry()

    def close_all_open_position(self):
        for p in self.getActivePositions():
            if p.isOpen():
                p.exitMarket()

In [10]:
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.bar import Frequency
import time
feed = csvfeed.GenericBarFeed(frequency=5*Frequency.SECOND)
feed.addBarsFromCSV("USD_JPY", "../data/candles/USD_JPY_S5_2020.csv")
order_book_json = open("../data/order_book/USD_JPY_OB_2020.json")
order_book = json.load(order_book_json)
myStrategy = NU3B(feed, "USD_JPY", order_book)
start = time.time()
myStrategy.run()
elapsed_time = time.time() - start
print(elapsed_time)

153.77074098587036
